<a href="https://colab.research.google.com/github/CapArrow/HighCountingVQA/blob/main/create_dataset_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import requests
import os
import zipfile
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
from imutils import paths
from tabulate import tabulate
import zipfile

In [2]:
# mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# change directory to the Colab Notebooks folder with link to drive of lars
%cd /content/drive/My Drive/MMDL2024/Colab Notebooks/

/content/drive/.shortcut-targets-by-id/1LQRkg9gduygZjO84k_knGQGpy1qEPTp6/Colab Notebooks


In [4]:
# load json
train_file = open("tallyqa_train.json")
test_file = open("tallyqa_test.json")
train_data = np.array(json.load(train_file))
test_data = np.array(json.load(test_file))
train_file.close()
test_file.close()

# test file has one additional key: "issimple"
# in the following this key is ignored
print(train_data[0].keys())
print(test_data[0].keys())


# concatenate data
data_dict = np.concatenate((train_data, test_data), axis=0)
# get dict keys
data_keys = data_dict[0].keys()
# transform dicts to list and get data array
df = pd.json_normalize(data_dict)
data = df.to_numpy()[:, :6]

dict_keys(['image', 'answer', 'data_source', 'question', 'image_id', 'question_id'])
dict_keys(['image', 'answer', 'data_source', 'question', 'image_id', 'question_id', 'issimple'])


In [5]:
def file_split_by_name(data):
  coco_train = []
  coco_val = []
  vg_100 = []
  vg_100_2 = []
  unknown = []
  for fname in data:
    foldern, imagen = fname.split("/")
    if foldern == "train2014":
      coco_train.append(fname)
    elif foldern == "val2014":
      coco_val.append(fname)
    elif foldern == "VG_100K":
      vg_100.append(fname)
    elif foldern == "VG_100K_2":
      vg_100_2.append(fname)
    else:
      unknown.append(fname)

  table = [["coco_train", len(coco_train)], ["coco_val", len(coco_val)], ["vg_100", len(vg_100)], ["vg_100_2", len(vg_100_2)], ["unknown", len(unknown)]]
  print(tabulate(table))
  return coco_train, coco_val, vg_100, vg_100_2

# extract all relevant image_names from data
data_image_names = np.unique(data[:, 0])
np.random.shuffle(data_image_names)
print(tabulate([["questions", len(data)], ["images", len(data_image_names)]]))
# get all images names wrt. the datasets which are used in tallyqa
coco_train_files, coco_val_files, vg_100k_files, vg_100k_2_files = file_split_by_name(data_image_names)

---------  ------
questions  287907
images     159432
---------  ------
----------  -----
coco_train  66943
coco_val    32633
vg_100      35270
vg_100_2    24586
unknown         0
----------  -----


In [ ]:
def download(url, zip_dir):
  if not os.path.exists(zip_dir):
      !wget -O $zip_dir $url

# coco
# download("http://images.cocodataset.org/zips/train2014.zip", "coco/train2014.zip")
download("http://images.cocodataset.org/zips/val2014.zip", "val2014.zip")
# download("http://images.cocodataset.org/zips/test2015.zip", "test2015.zip")

# visual genome
# download("https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip", "visual_genome/VG_100K.zip")
# download("https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip", "visual_genome/VG_100K_2.zip")

In [ ]:
%cd ..
!unzip val2014.zip -d coco/

In [ ]:
zip_files = ["val2014.zip"] # "train2014.zip", "VG_100K.zip", "VG_100K_2.zip"]

for file_name in zip_files:
  zf = zipfile.ZipFile(file_name)
  print(len(zf.namelist()))

  base_folder = ""
  folder_name = file_name.split(".zip")[0]
  tq_file_names = []

  if folder_name == "train2014":
    tq_file_names = coco_train_files
    base_folder = "coco/"
  elif folder_name == "val2014":
    tq_file_names = coco_val_files
    base_folder = "coco/"
  elif folder_name == "VG_100K":
    tq_file_names = vg_100k_files
    base_folder = "visual_genome/"
  else:
    tq_file_names = vg_100k_2_files
    base_folder = "visual_genome/"

  # extract all files seperately
  for fname in tqdm(tq_file_names):
    zf.extract(fname, base_folder)

In [6]:
print(len(list(paths.list_files("coco/train2014/"))))
print(len(list(paths.list_files("coco/val2014/"))))
print(len(list(paths.list_files("visual_genome/VG_100K/"))))
print(len(list(paths.list_files("visual_genome/VG_100K_2"))))

0
3086
0
0


In [ ]:
%cd coco/
coco_train_files = np.array(list(paths.list_files("train2014")))
coco_val_files = np.array(list(paths.list_files("val2014")))
%cd ../visual_genome/
vg_100_files = np.array(list(paths.list_files("VG_100K")))
vg_100_2_files = np.array(list(paths.list_files("VG_100K_2")))
%cd ..

In [ ]:
#mask_coco_train = np.isin(coco_train_files[:1000], data_image_names)
#mask_coco_val = np.isin(data_image_names, coco_val_files)
#mask_vg_100 = np.isin(data_image_names, vg_100_files)
#mask_vg_100_2 = np.isin(data_image_names, vg_100_2_files)

In [ ]:
filenotfound = []

for name in tqdm(data_image_names):
  folder, image_name = name.split("/")
  if folder == "VG_100K" or folder == "VG_100K_2":
    name = "visual_genome/" + name
  else:
    name = "coco/" + name
  if os.path.isfile(name):
    filenotfound.append(name)

In [ ]:
file_split_by_name(filenotfound)
print(str(len(filenotfound)) + " files not found!")
print(np.array(filenotfound)[:10])

139117 files not found!
['coco/train2014/COCO_train2014_000000401969.jpg'
 'coco/val2014/COCO_val2014_000000562879.jpg'
 'visual_genome/VG_100K/2316650.jpg' 'visual_genome/VG_100K/2358729.jpg'
 'visual_genome/VG_100K_2/2390799.jpg'
 'visual_genome/VG_100K_2/2413673.jpg'
 'coco/train2014/COCO_train2014_000000495903.jpg'
 'visual_genome/VG_100K/2343969.jpg'
 'coco/train2014/COCO_train2014_000000160573.jpg'
 'coco/train2014/COCO_train2014_000000274801.jpg']


In [9]:
# extract every question with answer greater then 4
gcd = data[:, 1] >= 4
greater_count_data = data[gcd]
lesser_count_data = data[np.logical_not(gcd)]
np.random.shuffle(lesser_count_data)
# create new data array with all answers greater then 4 and 20.000 answers lower then 4
new_data = np.concatenate((greater_count_data, lesser_count_data[:20000]), axis=0)
# train, val, test split (60, 20, 20)
np.random.shuffle(new_data)
max_len = len(new_data)
b1 = int(max_len * 0.6)
b2 = int(max_len * 0.8)
new_train_data = new_data[:b1]
new_val_data = new_data[b1:b2]
new_test_data = new_data[b2:]

In [18]:
def create_json(jdata, file_name):
  # create new json file for new_data
  file = open(file_name, "w")
  json_data = []

  for line in tqdm(jdata):
    tmp_dict = dict()
    if len(line[3]) != 0:
      for i, key in enumerate(data_keys):
          tmp_dict[key] = line[i]
    else:
      continue
    json_data.append(tmp_dict)

  json.dump(json_data, file)

In [19]:
create_json(new_train_data, "HighCountVQA_train.json")
create_json(new_val_data, "HighCountVQA_val.json")
create_json(new_test_data, "HighCountVQA_test.json")

100%|██████████| 12306/12306 [00:00<00:00, 311273.77it/s]
